In [9]:
import torch
import torch.nn as nn
from functools import partial
from collections import OrderedDict

In [10]:
def drop_path(x, drop_prob: float = 0, training: bool = False):
    if drop_prob == 0. or not training:
        return x
    keep_prob = 1 - drop_prob
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)
    random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.divice)
    random_tensor.floor_()
    output = x.div(keep_prob) * random_tensor
    return output

In [11]:
class DropPath(nn.Module):
    def __init__(self, drop_path=None):
        super(DropPath, self).__init__()
        self.drop_path = drop_path

    def forward(self, x):
        return drop_path(x, self.drop_path, self.training)



In [12]:
class PatchEmbed(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768, norm_layer=None):
        super().__init__()
        img_size = (img_size, img_size)
        patch_size = (patch_size, patch_size)
        self.img_size = img_size
        self.patch_size = patch_size
        self.gird_size = (img_size[0]//patch_size[0], img_size[1]//patch_size[1])
        self.num_patches = self.gird_size[0]*self.gird_size[1]

        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.norm = norm_layer(embed_dim) if norm_layer else nn.Identity()

    def forward(self,x):
        B, C, H, W = x.shape
        assert H == self.img_size[0] and W == self.img_size[1],\
        f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})"
        x = self.proj(x).flatten(2).transpose(1,2)
        x = self.norm(x)
        return x   
              


In [13]:
class Attention(nn.Module):
    def __init__(self,
                 dim,
                 num_heads = 8,
                 qkv_bias = False,
                 qk_scale = None,
                 atte_drop_ration = 0.,
                 proj_drop_ration = 0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5
        self.qkv = nn.Linear(dim, dim * 3, bias = qkv_bias)
        self.att_drop = nn.Dropout(atte_drop_ration)
        self.proj = nn.Dropout(proj_drop_ration)
        self.proj = nn.Linear(dim, dim)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C//self.num_heads).permute(2, 0,3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]
        attn = (q @ k. tanspose(-2,-1))*self.scale
        attn = attn.softmax(dim = -1)
        x = (attn @ v). transpose(1,2). reshape(B, N,C)
        x = self.porj(x)
        x= self.proj_srop(x)

        return x



    

In [14]:
class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fcl = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self,x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x
    
class Block(nn.Module):
    def __init__(self,
                 dim,
                 num_heads,
                 mlp_ratio = 4.,
                 qkv_bias = False,
                 qkv_scale = None,
                 drop_ratio = 0.,
                 attn_drop_ratio = 0.,
                 drop_path_ratio = 0.,
                 act_layer = nn.GELU,
                 norm_layer = nn.LayerNorm):
        super(Block,self).__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(dim, num_heads=num_heads,qkv_bias=qkv_bias,qkv_scale=qkv_scale,
                              attn_drop_ratio=attn_drop_ratio,proj_drop_ration=drop_ratio)
        self.drop_path = DropPath(drop_path_ratio) if drop_path_ratio>0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim*mlp_ratio)
        self.mlp =Mlp(in_features=dim,hidden_features=mlp_hidden_dim,act_layer=act_layer,drop=drop_ratio)


    def forward(self, x):
        x = x + self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x
